In [ ]:
import numpy                            as np
from   scipy           import          linalg
from   scipy.io        import         loadmat
from   scipy.integrate import          odeint
from   scipy.special   import   eval_legendre
from   scipy.special   import eval_gegenbauer

In [ ]:
# Finalizing Quadrature Points and Weight Functions

s = np.array(loadmat('s200.mat')['x']).flatten()
w = np.array(loadmat('w200.mat')['w']).flatten()

# Defining Trial and Test Functions (uj,vi)

def vi(v,arr):
    return np.sqrt(v+1.5)*(((eval_legendre(v+3, arr)-eval_legendre(v+1, arr))/((2*v+3)*(2*v+5)))-\
                           ((eval_legendre(v+1, arr)-eval_legendre(v-1, arr))/((2*v+1)*(2*v+3))))

def vid(v,arr):
    return (eval_legendre(v+2, arr)-eval_legendre(v, arr))/np.sqrt(2*(2*v+3))

def vidd(v,arr):
    return np.sqrt((2*v+3)/2)*eval_legendre(v+1, arr)

def uj(u,arr):
    return (eval_legendre(u+2, arr)-eval_legendre(u, arr))/(np.sqrt(2*(2*u+3)))
            
def ujd(u,arr):
    return np.sqrt((2*u+3)/2)*eval_legendre(u+1, arr)

um1  =                (1.+ s)/2 
um1d =                     1./2      
uz   = (-s**2/4) + (s/2) + 3./4
uzd  =              -s/2 + 1./2


# Functions to Numerically Obtain the Slope and Its Derivatives for Generic Values of 'm' and 'n'

def model(y,st,mt,nt,CIt):
    return (1./16)*(1-(CIt*((st+1)/4)**mt*(4*y)**nt))

def slope_and_derivatives(m_g,n_g,CI_g,s_g):   
    y0  =                                        0
    s_n =              np.append(np.array([-1]),s) 
    y   = odeint(model,y0,s_n,args=(m_g,n_g,CI_g))
    y   =                          y[1:].flatten() 
    yd  =                   np.zeros(s_g.shape[0])
    
    for i in range(s_g.shape[0]):
        yd[i] = (1./16)*(1-(CI_g*((s[i]+1)/4)**m_g*(4*y[i])**n_g)) 
        
    ydd = np.gradient(yd,s_g)
        
    return y, yd, ydd

In [ ]:
# Selecting Values of CI for the Run

CI_l = np.linspace(1,100,300)

m_sp  =   1.0            # Specific Drainage Area Exponent 
n_sp  =   1.0            # Local Slope Exponent
Ke    =  1e-4            # Fluvial Erosion Coefficient
L     =   100            # Domain Length-scale (Double of the Hillslope Length)
n     =    20            # Number of Modes (Trial and Test Functions)
U     =  1e-4            # Uplift Rate 

kval  =                     np.linspace(0.0035, 0.35, 100)  # Array of Wavenumbers 
dval  = ((Ke*L**(m_sp+n_sp))/(CI_l*U**(1-n_sp)))**(1/n_sp)  # Array of Soil Creep Coefficients
kmax  =                                      kval.shape[0]  # Count of Wavenumbers Array
dmax  =                                      dval.shape[0]  # Count of Creep Coefficients Array  

print("PLEASE CHECK, the value of m exponent taken is ", m_sp)
print("PLEASE CHECK, the value of n exponent taken is ", n_sp,'\n')

# Initializing Global Variables
sigmaOUT  =            np.zeros([dmax,kmax])
kout      =            np.zeros([dmax,kmax])
CIOUT     =            np.zeros([dmax,kmax])
ALLphiout = np.zeros([dmax*kmax,s.shape[0]])
ALLpsiout = np.zeros([dmax*kmax,s.shape[0]])

print("Channelization Indices used in this run are from", CI_l[0],  "to", CI_l[-1], '\n')
print(                   "The loop below would run for ", kmax+kmax*dmax,       " times")

In [ ]:
for dd in range(dmax):
    
    for kk in range(kmax):

        k    = kval[kk]                                   # Wavenumber for a Given Run
        khat =      k*L                                   # Dimensionless Wavenumber for a Given Run
        D    = dval[dd]                                   # Diffusion Coefficient for a Given Run 
        CI   = (Ke*L**(m_sp+n_sp))/(D**n_sp*U**(1-n_sp))  # Channelization Index for a Given Run
        
        
        # a-values
        
        a1   =           (n_sp*CI)/(4**(m_sp-n_sp))
        a2   =                              khat**2 
        a3   = (m_sp*CI*khat**2)/(4**(2+m_sp-n_sp))
        

        # Non-dimensional Slope and its Derivatives for the CI value of a Given Run
        
        So,Sod,Sodd = slope_and_derivatives(m_sp,n_sp,CI,s)
        

        # Gamma Values (Coefficients of the Eigen-value Problem)
        
        g1   = 16*So**2*(1+s)**2
        g2   = -32*So**2*(1+s) + a1*So**(n_sp+1)*(1+s)**(2+m_sp) + 32*So*Sod*(1+s)**2 
        g3   = 16*So*Sodd*(1+s)**2 - 32*So*Sod*(1+s) + 32*So**2 - a2*So**2*(1+s)**2 - a1*(1+s)**(m_sp+1)*So**(n_sp+1) + \
        a1*So**n_sp*Sod*(1+s)**(m_sp+2)
        g4   = a3*(So)**(1+n_sp)*(1+s)**(2+m_sp)
        g5   = (So*(1+s))**2
        

        # Gamma Derivative Values
        
        g1d  = 32*(1+s)*So*(So + (1+s)*Sod)        
        g1dd = 32*(So**2 + (1+s)**2 *Sod**2 + (1+s)*So*(4*Sod + (1+s)*(Sodd)))
        g2d  = -32*So**2 + a1*(2+m_sp)*(1+s)**(1+m_sp)*So**(1+n_sp) + a1*(1+n_sp)*(1+s)**(2+m_sp)*(So**n_sp)*Sod \
        + 32*(1+s)**2*(Sod**2) + 32*(1+s)**2*So*Sodd
        

        # Initializing RHS & LHS Matrix Elements
        
        VM       =           np.zeros(n)
        V        =           np.zeros(n)
        VMR      =           np.zeros(n)
        VR       =           np.zeros(n)
        pvett    =           np.zeros(n)
        qvett    =           np.zeros(n)
        COREA    =       np.zeros([n,n])
        COREB    =       np.zeros([n,n])
        alpha0   =                     0
        alpham1  =                     0
        phiout   =  np.zeros(s.shape[0])
        psiout   =  np.zeros(s.shape[0])

    
        # First Two Columns Corresponding to the LHS and RHS Matrices Before Imposing Boundary Conditions in the Strong Form
        
        for itri in range(n):
            i        = 1 + itri
            VM[itri] = np.sum((um1d*(g1dd*vi(i,s) + 2*g1d*vid(i,s) + g1*vidd(i,s) - g2d*vi(i,s)\
                                     - g2*vid(i,s) + g3*vi(i,s)) + (um1*g4*vi(i,s)))*w)

            V[itri]  = np.sum((uzd*(g1dd*vi(i,s) + 2*g1d*vid(i,s) + g1*vidd(i,s) - g2d*vi(i,s)\
                                     - g2*vid(i,s) + g3*vi(i,s)) + (uz*g4*vi(i,s)))*w)

        for itri in range(n):
            i         = 1 + itri
            VMR[itri] = np.sum(um1d*g5*vi(i,s)*w)
            VR[itri]  = np.sum( uzd*g5*vi(i,s)*w)
            
            
        # Core of the LHS(COREA) and RHS(COREB) Matrices Before Imposing Boundary Conditions in the Strong Form
                
        for itrj in range(0,n):
            for itri in range(0,n):
                j                = 1 + itrj
                i                = 1 + itri
                COREB[itri,itrj] = np.sum(ujd(j,s)*g5*vi(i,s)*w)
                COREA[itri,itrj] = np.sum((ujd(j,s)*(g1dd*vi(i,s) + 2*g1d*vid(i,s) + g1*vidd(i,s) - g2d*vi(i,s)\
                                     - g2*vid(i,s) + g3*vi(i,s)) + (uj(j,s)*g4*vi(i,s)))*w)        
                
                
        # Imposing Boundary Conditions in Strong Form and Updating the Matrices (A to A'; B TO B')
        
        for itri in range(n):
            i           = 1 + itri
            pvett[itri] = -np.sqrt((2*i+3)/2)*eval_gegenbauer(i,3/2,-1)/(-1/2)
            qvett[itri] = -np.sqrt((2*i+3)/2)*eval_legendre(1+i,1)/(1/2)   

        Asum   =    np.outer(V,pvett)
        Asum2  =   np.outer(VM,qvett)
        Bsum   =   np.outer(VR,pvett)
        Bsum2  =  np.outer(VMR,qvett)
        ADASH  = COREA + Asum + Asum2
        BDASH  = COREB + Bsum + Bsum2


        # Solving the Final Form of the Eigen-value Problem and Following the Maximum Real Part of the Eigenvalue
        
        Deig,      Veig =      linalg.eig(ADASH, BDASH)
        sigmaOUT[dd,kk] = np.max(Deig[:].real)*(D/L**2)
        

        # Tracing the Highest Value of Sigma and its Corresponding Eigenvector
        
        M,I            = np.max(Deig[:].real)*(D/L**2), np.argmax(Deig[:].real)
        kout[dd,kk]    =                                               kval[kk]
        CIOUT[dd,kk]   =                                                     CI
        veigcrit       =                                              Veig[:,I]

        
        # Obtaining Back alpha0 and alpham1 values; Building phi(s) and psi(s)
        
        for jitr in range(n):
                    alpha0  =  alpha0 + pvett[jitr]*veigcrit[jitr]
                    alpham1 = alpham1 + qvett[jitr]*veigcrit[jitr]


        for i in range(s.shape[0]):
            trialsA   =  np.concatenate((np.array([(-s[i]**2/4)+(s[i]/2)+3./4,(1.+s[i])/2]), [uj(j+1,s[i]) for j in range(n)]), axis=0) 
            trialsB   =  np.concatenate((np.array([ -s[i]/2+1./2,1./2]),                    [ujd(j+1,s[i]) for j in range(n)]), axis=0)
            coeff     =  np.append([alpha0.real, alpham1.real], veigcrit.real)
            phiout[i] =  np.sum(coeff*trialsA)
            psiout[i] = -np.sum(coeff*trialsB)*((64*So[i])/(khat**2*(s[i]+1)))

        ALLphiout[kk+kmax*dd,:] = phiout
        ALLpsiout[kk+kmax*dd,:] = psiout

In [ ]:
for i in range(CI_l.shape[0]):
    
    if (np.max(sigmaOUT[i,:])>0 and kout[i,np.argmax(sigmaOUT[i,:])] >0):
        
        # Extracting the Row (CI Value) for which First Positive Growth Rate at Some Wavenumber is Encountered
        i_cr     = i
        
        # Extracting the Wavenumber Curve for the CI_cr
        kcr_curve = kout[i,:]
        
        # Extracting the Sigma Curve for the CI_cr
        sig_curve = sigmaOUT[i,:]
        
        # Storing the CI_cr
        Ci_cr = CI_l[i]
        
        # Storing Critical Sigma for the CI_cr
        sig_cr = np.max(sigmaOUT[i,:])
        
        # Getting the Characterstic Wavenumber for the CI_cr
        K_cr   = kout[i,np.argmax(sigmaOUT[i,:])]
        
        # Getting the Characterstic Valley Spacing for the CI_cr  
        V_cr   = 2*np.pi/(kout[i,np.argmax(sigmaOUT[i,:])])
        
        print("For m=", m_sp, "and n=", n_sp,", CI_cr is ", Ci_cr, "and k_cr is ", K_cr, "based on the posed LSA problem \n")
        print("Run the Next Cell to Store the Relevant Arrays in the Current Directory")
        break

In [ ]:
np.save('./200_kout.npy',           kout)
np.save('./200_CI_l.npy',           CI_l)
np.save('./200_sigmaOUT.npy',   sigmaOUT)
np.save('./200_ALLphiout.npy', ALLphiout)
np.save('./200_ALLpsiout.npy', ALLpsiout)